##### Data in  "/data/shlok/data"

------------------------------------------------------------------------------------------------------------------------------------------------------------

In [72]:
import pandas as pd
import torch
df = pd.read_parquet('/data/shlok/data/full/001_Forbes.parquet')
# print(df.head(1))
# print(df.dtypes)
df.head(1)

,rank,personName,age,finalWorth,category,source,country,state,city,organization,selfMade,gender,birthDate,title,philanthropyScore,bio,about
0,1,Elon Musk,50.0,219000,Automotive,"Tesla, SpaceX",United States,Texas,Austin,Tesla,True,M,1971-06-28 00:00:00+00:00,CEO,1.0,Elon Musk is working to revolutionize transpor...,Musk was accepted to a graduate program at Sta...


In [73]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from table_bert import TableBertModel
model = TableBertModel.from_pretrained(
    '/data/ankur/TaBERT/tabert_large_k3/model.bin',
).to(device)

In [81]:
from table_bert import Table, Column
import pandas as pd
import torch

# Load your DataFrame
df = pd.read_parquet('/data/shlok/data/full/001_Forbes.parquet')

# Convert all categorical columns to object (string type)
for col in df.select_dtypes(include='category').columns:
    df[col] = df[col].astype(str)

# Now, fill NaNs and convert all data to strings
data = df.fillna('').astype(str).values.tolist()

# Function to map pandas dtypes to table_bert types
def map_dtype(dtype):
    if pd.api.types.is_numeric_dtype(dtype):
        return 'real'
    else:
        return 'text'

# Create header (list of Column objects) with sample values
header = []
for col in df.columns:
    dtype = map_dtype(df[col].dtype)
    sample_value = df[col].dropna().iloc[0]  # First non-null value as sample
    header.append(Column(col, dtype, sample_value=str(sample_value)))

# Create the Table object and tokenize it using the model's tokenizer
table = Table(
    id='001_Forbes',
    header=header,
    data=data
).tokenize(model.tokenizer)

# Define the context for the model
context = 'Name the richest person'

# Encode the context and table using the model
context_encoding, column_encoding, info_dict = model.encode(
    contexts=[model.tokenizer.tokenize(context)],
    tables=[table]
)


# input_ids = info_dict['tensor_dict']['input_ids']
# segment_ids = info_dict['tensor_dict']['segment_ids']
# context_token_positions = info_dict['tensor_dict']['context_token_positions']
# column_token_position_to_column_ids = info_dict['tensor_dict']['column_token_position_to_column_ids']
# sequence_mask = info_dict['tensor_dict']['sequence_mask']
# context_token_mask = info_dict['tensor_dict']['context_token_mask']
# table_mask = info_dict['tensor_dict']['table_mask']
# column_mask = info_dict['tensor_dict']['column_mask']
# instances = info_dict['instances']





In [77]:
outputs = model(input_ids,segment_ids,context_token_positions, column_token_position_to_column_ids, sequence_mask, context_token_mask, table_mask)


RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 3 and 2 at /tmp/pip-req-build-ocx5vxk7/aten/src/THC/generic/THCTensorMath.cu:62

In [80]:
info_dict

{'tensor_dict': {'input_ids': tensor([[[  101,  2171,  1996, 18429,  2711,   102,  4635,  1064,  2613,  1064,
             1015,   102,  2711, 18442,  1064,  3793,  1064,  3449,  2239, 14163,
             6711,   102,  2287,  1064,  2613,  1064,  2753,  1012,  1014,   102,
             2345,  5172,  1064,  2613,  1064, 20636,  8889,  2692,   102,  4696,
             1064,  3793,  1064, 12945,   102,  3120,  1064,  3793,  1064, 26060,
             1010,  2686,  2595,   102,  2406,  1064,  3793,  1064,  2142,  2163,
              102,  2110,  1064,  3793,  1064,  3146,   102,  2103,  1064,  3793,
             1064,  5899,   102,  3029,  1064,  3793,  1064, 26060,   102,  2969,
            21565,  1064,  2613,  1064,  2995,   102,  5907,  1064,  3793,  1064,
             1049,   102,  4182, 13701,  1064,  3793,  1064,  3411,  1011,  5757,
             1011,  2654,   102,  2516,  1064,  3793,  1064,  5766,   102, 29291,
             9363,  2890,  1064,  2613,  1064,  1015,  1012,  1014,   

### DEBUG
